# Install libraries, packages and dataset

In [ ]:
!git clone https://github.com/parth1620/Facial-Expression-Dataset.git
!pip install -U git+https://github.com/albumentations-team/albumentations
!pip install timm
!pip install --upgrade opencv-contrib-python

# Imports

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import torch

# Configurations

In [31]:
train_folder_path = 'Face-Expression-using-PyTorch/images/images/train'
validation_folder_path = 'Face-Expression-using-PyTorch/images/images/validation'

lr=0.0001
batches=32
epochs=10

device = 'cuda'
model_name = 'efficientnet_b0'

# Load Dataset

In [32]:
from torchvision.datasets import ImageFolder
from torchvision import transforms as T

In [33]:
train_augs = T.Compose([
   T.RandomHorizontalFlip(p=0.5),
   T.RandomRotation(degrees=(-20,20)),
   T.ToTensor()
])

valid_augs = T.Compose([
   T.ToTensor()
])

In [ ]:
trainset = ImageFolder(train_folder_path, transform=train_augs)
validset = ImageFolder(validation_folder_path, transform=valid_augs)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Facial-Expression-Dataset/images/train'

In [35]:
print(f"Total no. of examples in trainset : {len(trainset)}")
print(f"Total no. of examples in validset : {len(validset)}")

NameError: name 'trainset' is not defined

In [36]:
print(trainset.class_to_idx)

NameError: name 'trainset' is not defined

In [37]:
image, label = trainset[6390]

plt.imshow(image.permute(1,2,0))
plt.title(label)

NameError: name 'trainset' is not defined

# Load Dataset into Batches

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
trainloader = DataLoader(trainset, batch_size= batches, shuffle=True)
validloader = DataLoader(validset, batch_size= batches)

In [ ]:
print(f"Total no. of batches in trainloader : {len(trainloader)}")
print(f"Total no. of batches in validloader : {len(validloader)}")

In [ ]:
for images, labels in trainLoader:
   break

print(f"One image batch shape : {images.shape}")
print(f"One label batch shape : {labels.shape}")

# Create Model

In [ ]:
import timm
from torch import nn

In [ ]:
class FaceModel(nn.Module):
	def __init__(self):
		super(FaceModel, self).__init__()
		self.efficientnet = timm.create_model('efficientnet_b0', pretrained=True, num_classes=7)
      
	def forward(self, images, labels=None):
		logits = self.efficientnet(images)

		if labels != None:
			loss = nn.CrossEntropyLoss()(logits, labels)
			return logits, loss
		
		return logits

In [ ]:
model = FaceModel()
model.to(device)

# Create Train and Eval Function

In [ ]:
from tqdm import tqdm

In [ ]:
def multiclass_accuracy(y_pred,y_true):
    top_p,top_class = y_pred.topk(1,dim = 1)
    equals = top_class == y_true.view(*top_class.shape)
    return torch.mean(equals.type(torch.FloatTensor))

In [ ]:
def train_fn(model, dataloader, optimizer, current_epo):
	model.train()
	total_loss=0.0
	total_acc=0.0
	tk = tqdm(dataloader, desc = "EPOCH" + "[TRAIN]" + str(current_epo + 1) + "/`" + str(epochs))
   
	for t, data in enumerate(tk):
		images, labels =data
		images, labels = images.to(device), labels.to(device)

		optimizer.zero_grad()
		logits, loss = model(images, labels)
		loss.backwar()
		optimizer.step()

		total_loss+=loss.item()
		total_acc+=multiclass_accuracy(logits, labels)
		tk.set_postfix({'loss' : '%5f' %float(loss = total_loss/(t+1)), 'acc' : '%5f' %float(accuracy = total_acc/(t+1)),})

	return total_loss/len(dataloader), total_acc/len(dataloader)

In [ ]:
def eval_fn(model, dataloader, current_epo):
	model.eval()
	total_loss=0.0
	total_acc=0.0
	tk = tqdm(dataloader, desc = "EPOCH" + "[VALID]" + str(current_epo + 1) + "/`" + str(epochs))
   
	for t, data in enumerate(tk):
		images, labels =data
		images, labels = images.to(device), labels.to(device)

		logits, loss = model(images, labels)

		total_loss+=loss.item()
		total_acc+=multiclass_accuracy(logits, labels)
		tk.set_postfix({'loss' : '%5f' %float(loss = total_loss/(t+1)), 'acc' : '%5f' %float(accuracy = total_acc/(t+1)),})

	return total_loss/len(dataloader), total_acc/len(dataloader)

# Create Training Loop

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
bet_valid_loss = np.Inf

for i in range(epochs):
	train_loss, train_acc = train_fn(model, trainloader, optimizer, i)
	valid_loss, valid_acc = eval_fn(model, validloader, i)
   
	if valid_loss < bet_valid_loss:
		torch.save(model.state_dict(), 'best-weights.pth')
		print("Saved best weights")
		bet_valid_loss = valid_loss

# Inference

In [ ]:
def view_classify(img, ps):

    classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

    ps = ps.data.cpu().numpy().squeeze()
    img = img.numpy().transpose(1,2,0)

    fig, (ax1, ax2) = plt.subplots(figsize=(5,9), ncols=2)
    ax1.imshow(img)
    ax1.axis('off')
    ax2.barh(classes, ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(classes)
    ax2.set_yticklabels(classes)
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()

    return None